# AI_A3
## Kasra Ghorbani
### 810101489
#### clustering

The goal here is to first extract features from a set of 210 pre-processed photos of different flowers and assign photos to different clusters based on the those features. The steps include :

1- Accessing the image folder and iterating over all photos
2- In each iteration preprocess photos so that they're better suited for feature extraction
3- In each iteration extract features from photos using existing libraries
4- Append the extracted features to the list off all features we've gathered.


In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans, DBSCAN
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.models import Model
from sklearn.metrics import adjusted_rand_score
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score, homogeneity_score


base_model = VGG16(weights='imagenet', include_top=False)
output = base_model.output
feature_extraction_model = Model(inputs=base_model.input, outputs=output)

csv_file = 'flower_labels.csv'
df = pd.read_csv(csv_file)
labels = df['label'].values

all_features = []

images_dir = 'C:\\Users\\a\\Desktop\\AI-A3\\flower_images'
for filename in os.listdir(images_dir):
    img_path = os.path.join(images_dir, filename)
    img = image.load_img(img_path, target_size=(224, 224))  # resize
    x = image.img_to_array(img)  # save features as numpy arr
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)

    features = feature_extraction_model.predict(x)
    all_features.append(features.reshape(-1))  # Flatten features

all_features = np.array(all_features)

ModuleNotFoundError: No module named 'cv2'

Next we need to choose a K for the K-means algorithm and an epsilon and min samples for thr DBscan algorithm.
Through experimenting with different values for all of these 3 variables I chose the values below in the code.
Then we form an "evaluation df" which contains 3 columns with the true label, and the cluster we chose for each node in each algorithm. And alast we reduce the dimentionality of the features to make them observanle in 2D space.

In [ ]:
num_clusters = 10
kmeans = KMeans(n_clusters=num_clusters)
cluster_labels_kmeans = kmeans.fit_predict(all_features)

dbscan = DBSCAN(eps=1400, min_samples=2)
cluster_labels_dbscan = dbscan.fit_predict(all_features)

evaluation_df = pd.DataFrame({'label': labels,
                              'cluster_kmeans': cluster_labels_kmeans,
                              'cluster_dbscan': cluster_labels_dbscan})

pca = PCA(n_components=2)
reduced_features = pca.fit_transform(all_features)
evaluation_df['PCA1'] = reduced_features[:, 0]
evaluation_df['PCA2'] = reduced_features[:, 1]

Afterwards we plot the clusterings done by Kmeans and DBscan. "unique_clusters_dbscan = np.unique(cluster_labels_dbscan)" is included to handle the dynamic assignment of cluster labels by DBSCAN, which may result in non-consecutive cluster labels, including noise points labeled as -1.

In [ ]:
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
for cluster in range(num_clusters):
    cluster_data = evaluation_df[evaluation_df['cluster_kmeans'] == cluster]
    plt.scatter(cluster_data['PCA1'],
                cluster_data['PCA2'], label=f'Cluster {cluster}')
plt.title('K-means Clustering Visualization using PCA')
plt.xlabel('PCA Component 1')
plt.ylabel('PCA Component 2')
plt.legend()

plt.subplot(1, 2, 2)
unique_clusters_dbscan = np.unique(cluster_labels_dbscan)
for cluster in unique_clusters_dbscan:
    if cluster == -1:  # Noise points
        cluster_data = evaluation_df[evaluation_df['cluster_dbscan'] == cluster]
        plt.scatter(cluster_data['PCA1'], cluster_data['PCA2'], label=f'Noise')
    else:
        cluster_data = evaluation_df[evaluation_df['cluster_dbscan'] == cluster]
        plt.scatter(cluster_data['PCA1'],
                    cluster_data['PCA2'], label=f'Cluster {cluster}')
plt.title('DBSCAN Clustering Visualization using PCA')
plt.xlabel('PCA Component 1')
plt.ylabel('PCA Component 2')
plt.legend()

plt.show()

Next we've shown the elbow method based on the squared sum of distances within clusters for Kmeans clustering.

In [ ]:
wcss = []  # Within-cluster sum of squares
for k in range(1, 11):
    kmeans = KMeans(n_clusters=k)
    kmeans.fit(all_features)
    wcss.append(kmeans.inertia_)

plt.figure(figsize=(8, 6))
plt.plot(range(1, 11), wcss, marker='o', linestyle='-', color='b')
plt.title('Elbow Method for Optimal k')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Within-Cluster Sum of Squares (WCSS)')
plt.xticks(range(1, 11))
plt.grid(True)
plt.show()

In the end we've measured our clustering performance based on the Homogeneity and silhouette methods.

In [ ]:
silhouette_avg_kmeans = silhouette_score(all_features, cluster_labels_kmeans)
print(f"For KMeans, the average silhouette_score is : {silhouette_avg_kmeans}")

homogeneity_kmeans = homogeneity_score(labels, cluster_labels_kmeans)
print(f"For KMeans, the homogeneity_score is : {homogeneity_kmeans}")

silhouette_avg_dbscan = silhouette_score(all_features, cluster_labels_dbscan)
print(f"For DBSCAN, the average silhouette_score is : {silhouette_avg_dbscan}")

homogeneity_dbscan = homogeneity_score(labels, cluster_labels_dbscan)
print(f"For DBSCAN, the homogeneity_score is : {homogeneity_dbscan}")

For KMeans, the average silhouette_score is : 0.03699790686368942
For KMeans, the homogeneity_score is : 0.4529131687889035
For DBSCAN, the average silhouette_score is : -0.02012842334806919
For DBSCAN, the homogeneity_score is : 0.09380551835749275


## Questions 
#### 1-
Feature extraction is crucial because raw pixel values don't convey semantic information. Extracted features capture meaningful patterns like edges and textures, enabling machine learning models to learn effectively.
#### 2-
Common feature extraction techniques for images include Convolutional Neural Networks (CNNs), which learn hierarchical representations, Histogram of Oriented Gradients (HOG), which captures shape and edge information, and Scale-Invariant Feature Transform (SIFT), which detects and describes local features invariant to scale and rotation.
#### 3-
Image preprocessing steps, such as resizing for uniformity, normalization for convergence during training, and data augmentation for increasing diversity and robustness, are essential for preparing images as model inputs.
#### 4-
K-means:
Advantages:
Simple implementation and fast execution.
Ability to handle large volumes of data efficiently.
Disadvantages:
Sensitivity to the number of clusters (K).
Dependence on the initial cluster centroids, which may lead to different results.
DBSCAN:
Advantages:
Ability to identify clusters of arbitrary shapes and sizes.
Robustness to noise and ability to detect outliers.
Disadvantages:
Sensitivity to parameters such as eps and min_samples.
Need for parameter tuning for optimal performance.
#### 5-
I experimented with different K values on random states. I also drew the elbow graph for within-cluster squared distance sum and also looked at the placement of the dots and decided on the number i used besides checking the results for silhouette and homogeneity.
#### 6-
DBscan doesn't perform well here for some reason in comparison to Kmeans.
#### 7-
Principal Component Analysis (PCA) is a dimensionality reduction technique used to simplify datasets while preserving important information. It works by transforming the original features into a new set of uncorrelated variables called principal components. These components are ordered by the amount of variance they explain in the data, with the first component explaining the most variance and subsequent components explaining decreasing amounts. By retaining only the top principal components, PCA reduces the dimensionality of the dataset while minimizing information loss. This helps in visualizing high-dimensional data, removing redundant features, and speeding up machine learning algorithms.
#### 8-
Silhouette Score:
The silhouette score measures how similar an object is to its own cluster compared to other clusters.
For each sample, it computes the average distance from that sample to all other points in the same cluster (a) and the average distance to all points in the nearest cluster (b).
The silhouette score (s) for a single sample is then given by (b - a) / max(a, b).
The silhouette score for the entire dataset is the mean of the silhouette scores for each sample, ranging from -1 to 1, where a higher value indicates better clustering.
Homogeneity Score:
The homogeneity score measures the homogeneity of the clusters, meaning each cluster contains only members of a single class.
It computes how much each cluster contains only points which are members of a single class, ignoring the class labels assigned to the samples.
The homogeneity score is a value between 0 and 1, where 1 indicates perfectly homogeneous labeling.
#### 9-
The results are shown under the related code.
#### 10-
Experimenting more values for DBscan and Kmeans, better preprocessing to help us get rid of redundant information which only decrease our accuracy, Using a bigger and more diverse data set perhaps, We can perhaps use different pre-trained models for image processing.